In [11]:
"""
    Debugging Notes: 
    
    Change approach: Have penalty multiplier based on violations instead of hard filtering
    
    
    To Do List:
    Sensitivity Analysis for penalty multipliers
    
    Modify objective function
    Implement self generated energy for overenergylimit() and obj_func()
        >Just change consumption to consumption - selfproduced
        >selfproduced for cost will be multiplied to offcost
"""



# Import modules
import numpy as np

# Import sphere function as objective function
from pyswarms.utils.functions.single_obj import sphere as f

# Import backend modules
import pyswarms.backend as P
from pyswarms.backend.topology import Star
#import pyswarms as ps
#import pyswarms.discrete.binary as PSO

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
my_topology = Star() # The Topology Class
my_options = {'c1': 1, 'c2': 1, 'w': 1}
c1 = 1
c2 = 1
w = 1
#set dimension to 10 loads x 16 timeslots = 160
my_swarm = P.create_swarm(n_particles=10, dimensions=160, binary=True, discrete=True, options=my_options) # The Swarm Class

#Initialize cost to array of inf for max value, to_append is manually set below. Create temp storade for pbest cost as well
x = 0
temp_pbest_cost = []
to_append = np.array([float('inf')])

for x in range(my_swarm.n_particles):
    my_swarm.pbest_cost = np.append(my_swarm.pbest_cost, to_append)
    my_swarm.current_cost = np.append(my_swarm.current_cost, to_append)
    temp_pbest_cost = np.append(temp_pbest_cost, to_append)




#Printing Values for debugging
print('The following are the attributes of our swarm: {}'.format(my_swarm.__dict__.keys()))
print("# of particles:",my_swarm.n_particles)
print("dimensions:",my_swarm.dimensions)
print("pos \n",my_swarm.position)
print("velo \n",my_swarm.velocity)
print("current cost \n",my_swarm.current_cost)
print("pbest pos \n",my_swarm.pbest_pos)
print("pbest cost \n",my_swarm.pbest_cost)
print("best cost \n",my_swarm.best_cost)

#print(my_swarm.position[0][0:3])
#print(my_swarm.position[0][3:6])
#print(my_swarm.position[0][6:9])

The following are the attributes of our swarm: dict_keys(['position', 'velocity', 'n_particles', 'dimensions', 'options', 'pbest_pos', 'best_pos', 'pbest_cost', 'best_cost', 'current_cost'])
# of particles: 10
dimensions: 160
pos 
 [[0 1 0 ... 1 1 1]
 [0 1 1 ... 0 0 1]
 [1 1 0 ... 1 0 1]
 ...
 [1 0 1 ... 0 0 0]
 [1 1 1 ... 0 1 1]
 [0 1 1 ... 1 1 1]]
velo 
 [[0.74986066 0.32041079 0.90806067 ... 0.78256849 0.51614785 0.60758328]
 [0.71262806 0.88530223 0.06855021 ... 0.72823683 0.75120999 0.59044656]
 [0.14943681 0.55739531 0.90495979 ... 0.93048818 0.90148079 0.84850205]
 ...
 [0.74149103 0.39848085 0.067386   ... 0.55812231 0.74578328 0.59307383]
 [0.20934459 0.96500345 0.99828848 ... 0.26128342 0.33058503 0.34673213]
 [0.30496064 0.57786502 0.40851192 ... 0.58038996 0.85760303 0.46861264]]
current cost 
 [inf inf inf inf inf inf inf inf inf inf]
pbest pos 
 [[0 1 0 ... 1 1 1]
 [0 1 1 ... 0 0 1]
 [1 1 0 ... 1 0 1]
 ...
 [1 0 1 ... 0 0 0]
 [1 1 1 ... 0 1 1]
 [0 1 1 ... 1 1 1]]
pbest co

In [13]:
#Position computation, BPSO computation for position

def _compute_position(swarm):
    for i in range(swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            if _sigmoid(swarm.velocity[i][j]) < np.random.uniform(low=0.0, high=1.0):
                swarm.position[i][j] = 1
            else:
                swarm.position[i][j] = 0   
    return swarm.position

def _sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [14]:
#Velocity computation, BPSO computation for velocity

def _compute_velocity(swarm,c1,c2,w):
    for i in range(my_swarm.n_particles):                  
        for j in range(my_swarm.dimensions):
            swarm.velocity[i][j] = w*swarm.velocity[i][j] + \
            [c1*np.random.uniform(low=0.0, high=1.0)*swarm.pbest_pos[i][j]]-[c1*np.random.uniform(low=0.0, high=1.0)*swarm.position[i][j]] + \
            [c2*np.random.uniform(low=0.0, high=1.0)*my_swarm.best_cost]-[c2*np.random.uniform(low=0.0, high=1.0)*swarm.position[i][j]]
    return swarm.velocity

In [15]:
#Taken from pyswarms.backend.operators
def _compute_pbest(swarm,temp_pbest_cost):
    try:
        # Infer dimensions from positions
        dimensions = swarm.dimensions
        
        # Create a 1-D and 2-D mask based from comparisons
        # Creating 1-D mask
        mask_cost = swarm.current_cost < temp_pbest_cost
        #print("maskcost:\n",mask_cost)|
        
        # Extension of 1-D mask to 2-D mask
        mask_pos = np.repeat(mask_cost[:, np.newaxis], dimensions, axis=1)
        #print("maskpos:\n",mask_pos)
        
        # Apply masks, copy into 2nd variable if 3rd variable fits conditions
        new_pbest_pos = np.where(~mask_pos, swarm.pbest_pos, swarm.position)
        new_pbest_cost = np.where(~mask_cost, temp_pbest_cost, swarm.current_cost)
        
        #print("swarm current pos:",swarm.position)
        #print("new swarm pbest pos:",new_pbest_pos)
        #print("new swarm pbest cost:",new_pbest_cost)
        
    except AttributeError:
        rep.logger.exception(
            "Please pass a Swarm class. You passed {}".format(type(swarm))
        )
        raise
    else:
        return (new_pbest_pos, new_pbest_cost)

In [16]:
#penalty for curtailing more than necessary (lighter penalty)
def overcurtail(x):
    
    #timeslotrange = 160 elements per particle divided by 10 particles
    timeslotrange = 16
    penaltyvalue = 1000
    totalpenalty = 0
    
    #divide array into 10 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    
    #invert array 1 and 0, count 1s to find number of 0s
    inv = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    L0 = np.subtract(inv,L0)
    L1 = np.subtract(inv,L1)
    L2 = np.subtract(inv,L2)
    L3 = np.subtract(inv,L3)
    L4 = np.subtract(inv,L4)
    L5 = np.subtract(inv,L5)
    L6 = np.subtract(inv,L6)
    L7 = np.subtract(inv,L7)
    L8 = np.subtract(inv,L8)
    L9 = np.subtract(inv,L9)
    
    #set maximum off time per load here
    limitL0 = 8
    limitL1 = 8
    limitL2 = 8
    limitL3 = 8
    limitL4 = 8
    limitL5 = 8
    limitL6 = 8
    limitL7 = 8
    limitL8 = 8
    limitL9 = 8
    
    #count for off time here, add 1 penalty value if violated
    if np.sum(L0) > limitL0:
        totalpenalty += 1
    if np.sum(L1) > limitL1:
        totalpenalty += 1
    if np.sum(L2) > limitL2:
        totalpenalty += 1
    if np.sum(L3) > limitL3:
        totalpenalty += 1
    if np.sum(L4) > limitL4:
        totalpenalty += 1
    if np.sum(L5) > limitL5:
        totalpenalty += 1
    if np.sum(L6) > limitL6:
        totalpenalty += 1
    if np.sum(L7) > limitL7:
        totalpenalty += 1
    if np.sum(L8) > limitL8:
        totalpenalty += 1
    if np.sum(L9) > limitL9:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [17]:
#penalty for undercurtailing (should be high multiplier to ensure output is not all zero)
def undercurtail(x):
    
    #timeslotrange = 160 elements per particle divided by 10 particles
    timeslotrange = 16
    penaltyvalue = 1000000
    totalpenalty = 0
    
    #divide array into 10 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    
    #set minimum on time per load here
    limitL0 = 10
    limitL1 = 10
    limitL2 = 10
    limitL3 = 10
    limitL4 = 10
    limitL5 = 10
    limitL6 = 10
    limitL7 = 10
    limitL8 = 10
    limitL9 = 10
    
    #count for ontime here, add 1 penalty value if violated
    if np.sum(L0) < limitL0:
        totalpenalty += 1
    if np.sum(L1) < limitL1:
        totalpenalty += 1
    if np.sum(L2) < limitL2:
        totalpenalty += 1
    if np.sum(L3) < limitL3:
        totalpenalty += 1
    if np.sum(L4) < limitL4:
        totalpenalty += 1
    if np.sum(L5) < limitL5:
        totalpenalty += 1
    if np.sum(L6) < limitL6:
        totalpenalty += 1
    if np.sum(L7) < limitL7:
        totalpenalty += 1
    if np.sum(L8) < limitL8:
        totalpenalty += 1
    if np.sum(L9) < limitL9:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [18]:
#penalty for exceeding energy limit per timeslot
def overenergylimit(x):
    penaltyvalue = 100000
    totalpenalty = 0
    
    #set limit for each TS in kW here
    TS0limit = 8000
    TS1limit = 8000
    TS2limit = 8000
    TS3limit = 8000
    TS4limit = 8000
    TS5limit = 8000
    TS6limit = 8000
    TS7limit = 8000
    TS8limit = 8000
    TS9limit = 8000
    TS10limit = 8000
    TS11limit = 8000
    TS12limit = 8000
    TS13limit = 8000
    TS14limit = 8000
    TS15limit = 8000
    
    #set consumption for each load in kW here
    cons0 = 1000
    cons1 = 1000
    cons2 = 1000
    cons3 = 1000
    cons4 = 1000
    cons5 = 1000
    cons6 = 1000
    cons7 = 1000
    cons8 = 1000
    cons9 = 1000
    consumption = [cons0, cons1, cons2, cons3, cons4, cons5, cons6, cons7, cons8, cons9]
    
    #cut particles into timeslots here, from L0 to L15. Note: 160 = 10 loads x 16 timeslots
    TS0 = x[0:160:16]
    TS1 = x[1:160:16]
    TS2 = x[2:160:16]
    TS3 = x[3:160:16]
    TS4 = x[4:160:16]
    TS5 = x[5:160:16]
    TS6 = x[6:160:16]
    TS7 = x[7:160:16]
    TS8 = x[8:160:16]
    TS9 = x[9:160:16]
    TS10 = x[10:160:16]
    TS11 = x[11:160:16]
    TS12 = x[12:160:16]
    TS13 = x[13:160:16]
    TS14 = x[14:160:16]
    TS15 = x[15:160:16]
    
    #check for instances of overconsumption
    if np.sum(np.multiply(TS0,consumption)) > TS0limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS1,consumption)) > TS1limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS2,consumption)) > TS2limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS3,consumption)) > TS3limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS4,consumption)) > TS4limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS5,consumption)) > TS5limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS6,consumption)) > TS6limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS7,consumption)) > TS7limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS8,consumption)) > TS8limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS9,consumption)) > TS9limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS10,consumption)) > TS10limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS11,consumption)) > TS11limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS12,consumption)) > TS12limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS13,consumption)) > TS13limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS14,consumption)) > TS14limit:
        totalpenalty += 1
    if np.sum(np.multiply(TS15,consumption)) > TS15limit:
        totalpenalty += 1
    
    totalpenalty = totalpenalty*penaltyvalue
    
    return totalpenalty

In [19]:
#test function, 2D version, to be implemented per element (Element is 1D array)
def obj_func(x):
    
    #timeslotrange = 160 elements per particle divided by 10 particles
    timeslotrange = 16
    
    #set costs per kW here
    oncost = 10
    offcost = 5
    
    #set consumption for each load in kW here
    cons0 = 1000
    cons1 = 1000
    cons2 = 1000
    cons3 = 1000
    cons4 = 1000
    cons5 = 1000
    cons6 = 1000
    cons7 = 1000
    cons8 = 1000
    cons9 = 1000
    
    #divide array into 10 loads x 16 timeslots
    L0 = x[0:timeslotrange]
    L1 = x[timeslotrange:(timeslotrange*2)]
    L2 = x[(timeslotrange*2):(timeslotrange*3)]
    L3 = x[(timeslotrange*3):(timeslotrange*4)]
    L4 = x[(timeslotrange*4):(timeslotrange*5)]
    L5 = x[(timeslotrange*5):(timeslotrange*6)]
    L6 = x[(timeslotrange*6):(timeslotrange*7)]
    L7 = x[(timeslotrange*7):(timeslotrange*8)]
    L8 = x[(timeslotrange*8):(timeslotrange*9)]
    L9 = x[(timeslotrange*9):(timeslotrange*10)]
    
    f =  np.sum(np.array(L0)*oncost*cons0) + np.sum(np.array(L1)*oncost*cons1) + np.sum(np.array(L2)*oncost*cons2) + \
    np.sum(np.array(L3)*oncost*cons3) + np.sum(np.array(L4)*oncost*cons4) + np.sum(np.array(L5)*oncost*cons5) + \
    np.sum(np.array(L6)*oncost*cons6) + np.sum(np.array(L7)*oncost*cons7) + np.sum(np.array(L8)*oncost*cons8) + np.sum(np.array(L9)*oncost*cons9) +\
    overcurtail(x) + undercurtail(x) + overenergylimit(x)
    return f


In [20]:
#print("Initial Value current cost:\n",my_swarm.current_cost)
#print("Initial Value current pos:\n",my_swarm.position)
#print("Initial Value pbest pos:\n",my_swarm.pbest_pos)
#print("Initial Value pbest cost:\n",my_swarm.pbest_cost)

iterations = 100 # Set 100 iterations
for i in range(iterations):
    
    # Part 0: Compute current cost
    j=0
    for j in range(my_swarm.n_particles):
        my_swarm.current_cost[j] = obj_func(my_swarm.position[j][:]) # Compute current cost (using obj func)
        temp_pbest_cost[j] = obj_func(my_swarm.pbest_pos[j][:])  # Compute personal best cost (using obj func)
    
    #my_swarm.pbest_cost = f(my_swarm.pbest_pos) # Compute current cost (using built in sphere func)
    #print("-------------------------------")
    #print("Pbest pos:\n",my_swarm.pbest_pos)
    #print("Pbest cost:\n",my_swarm.pbest_cost)
    #print("Current pos:\n",my_swarm.position)
    #print("Current cost:\n", my_swarm.current_cost)
   
    #Update personal best pos/cost 
    my_swarm.pbest_pos, my_swarm.pbest_cost = _compute_pbest(my_swarm,temp_pbest_cost) # Update and store
    
    # Part 2: Update global best
    # Note that gbest computation is dependent on your topology
    if np.min(my_swarm.pbest_cost) < my_swarm.best_cost:
        my_swarm.best_pos, my_swarm.best_cost = my_topology.compute_gbest(my_swarm)

    # Let's print our output
    if i%20==0:
        print('Iteration: {} | my_swarm.best_cost: {:.4f}'.format(i+1, my_swarm.best_cost))

    # Part 3: Update position and velocity matrices
    # Note that position and velocity updates are dependent on your topology
    
    #Fix the velocity and position computation for Binary application
    
    my_swarm.velocity = _compute_velocity(my_swarm,c1,c2,w)
    my_swarm.position = _compute_position(my_swarm)   
    
print('The best cost found by our swarm is: {:.4f}'.format(my_swarm.best_cost))
print('The best position found by our swarm is: {}'.format(my_swarm.best_pos))

Iteration: 1 | my_swarm.best_cost: 5944000.0000
Iteration: 21 | my_swarm.best_cost: 5944000.0000
Iteration: 41 | my_swarm.best_cost: 5944000.0000
Iteration: 61 | my_swarm.best_cost: 5944000.0000
Iteration: 81 | my_swarm.best_cost: 5944000.0000
The best cost found by our swarm is: 5944000.0000
The best position found by our swarm is: [0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0
 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1
 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1
 1 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1
 0 0 0 1 1 0 1 1 1 1 1 1]


In [21]:
#Sample solution for counting 1s (This fix is functional)
#Import modules
import numpy as np

testarr = [[1, 0, 1, 0, 1, 1, 0, 0, 1],[1, 1, 1, 0, 1, 1, 1, 0, 1],[0, 0, 1, 0, 1, 0, 0, 0, 1]]
print("test array:",testarr)
print("number of arrays:",len(testarr))
print("length of array:",len(testarr[0]))

i=0
j=0

for i in range(len(testarr)):
    j = testarr[i].count(1)
    k = testarr[i].count(0)
    print("The 1s in array %i is %i" %(i,j))
    print("The 0s in array %i is %i" %(i,k))
    j=0
    k=0


test array: [[1, 0, 1, 0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 0, 0, 0, 1]]
number of arrays: 3
length of array: 9
The 1s in array 0 is 5
The 0s in array 0 is 4
The 1s in array 1 is 7
The 0s in array 1 is 2
The 1s in array 2 is 3
The 0s in array 2 is 6


In [28]:
arr0 = [1,2,3,4,5]
arr1 = [6,7,8,9,10]
arr2 = [11,12,13,14,15]
totalarr = [arr0, arr1, arr2]

sum = totalarr[0] + totalarr[1] + totalarr[2]
print(sum)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [39]:
import numpy as np
y = np.array([1, 2, 2, 2, 2, 0, 2, 3, 3, 3, 0, 0, 2, 2, 0])
print(np.count_nonzero(y == 1))


1


In [29]:
import numpy as np
array = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
arr1 = np.array(array[0:len(array):5])
arr2 = array[1:len(array):5]
arr3 = array[2:len(array):5]
arr4 = array[3:len(array):5]
arr5 = array[4:len(array):5]
num = 5

arr2 = np.array(arr1)*2*num
arr3 = arr1 + arr2
arr4 = np.sum(arr1)
arr5 = np.sum(np.multiply(arr1,arr2))
print("array 1:", arr1)
print("array 2:", arr2)
print("array 3:", arr3)
print("array 4:", arr4)
print("array 5:", arr5)

array 1: [ 1  6 11 16]
array 2: [ 10  60 110 160]
array 3: [ 11  66 121 176]
array 4: 34
array 5: 4140
